In [1]:
import sys
sys.path.append('../custom scripts/')

import multisession_analysis.multisession_registration as tracker

Using TensorFlow backend.


In [2]:
# Which sessions should be aligned?
session_list = [r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200818',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200819',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200820',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200821',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200824',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200826',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200827',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200830',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200902',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200905',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200908',
                r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200911']

# Which session should be the reference (place cells from this session will be tracked)
reference_session = '20200826'

In [4]:
# This function loads the data of all sessions and stores it in the list "pcf_objects"
spatial, templates, dim, pcf_objects = tracker.load_multisession_data(session_list)

Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200818\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200818 (1/12).
Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200819\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200819 (2/12).
Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200820\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200820 (3/12).
Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200821\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200821 (4/12).
Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200824\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200824 (5/12).
Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M38\20200826\pcf_results.pickle...
Successfully loaded data from Batch3\M38\20200826 (6/12).
Loading file W:\Neurop

In [5]:
# This function prepares the data for the tracking process
target_session_list, place_cell_indices, alignment_array, all_contours_list, all_shifts_list = tracker.prepare_manual_alignment_data(
                                                                                                          pcf_objects, reference_session)


In [ ]:
# If you started to align the place cells of this session, but saved the table incomplete and want to continue, you can
# load the table with this function and pick up where you left. Note that the file path should include the 
# file name and extension
file_path = r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\batch_processing\cell_alignments\pc_alignment_20200826.txt'
alignment_array = tracker.load_alignment(file_path)

In [6]:
# This is the main function for the tracking. It creates the interactive plot and saves the results in the
# alignment_array, which has one place cell for each row, one session in each column, and each entry is the 
# neuron ID that the reference cell has in each session.
%matplotlib qt
alignment_array = tracker.manual_place_cell_alignment(pcf_sessions=pcf_objects,
                                                      target_sessions=target_session_list,
                                                      cell_idx=place_cell_indices,
                                                      alignment=alignment_array,
                                                      all_contours=all_contours_list,
                                                      all_shifts=all_shifts_list,
                                                      ref_sess=reference_session,
                                                      dim=dim,
                                                      show_neuron_id=True)

Reference cell 0, clicked neuron (0, 0, 56).
Reference cell 0, clicked neuron (0, 1, 161).
Reference cell 0, clicked neuron (0, 2, 38).
Reference cell 0, clicked neuron (0, 3, 35).
Reference cell 0, clicked neuron (0, 4, 41).
Reference cell 0, clicked neuron (0, 6, 149).
Reference cell 0, clicked neuron (0, 7, 43).
Reference cell 0, clicked neuron (0, 8, -1).


In [8]:
# In case the correct cell is not displayed in the alignment plot, you can show the whole FOVs of the reference
# as well as the target session and look for the correct cell yourself. Use the indices displayed in the 
# interactive graph to access the correct session from the list and the correct cell ID.
tracker.show_whole_fov(reference_session=pcf_objects[5], target_session=pcf_objects[2], ref_cell_id=120)


In [7]:
# Save the alignment array under the provided directory as a csv table. Every row is one place cell from the 
# reference session, every column is one session, and the entries are the IDs of each cell in the corresponding
# session.
file_directory = r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\batch_processing\cell_alignments'
tracker.save_alignment(file_directory, alignment_array, reference_session, pcf_objects)


Saving alignment table to W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\batch_processing\cell_alignments\pc_alignment_20200826.txt...
